In [149]:
import reading
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dense, Flatten, InputLayer, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [150]:
dataset = pd.read_csv('ADNI_dataset_diffusion.csv')
dataset["Path"] = reading.data_path("Diffusion_parameters_maps-20230215T134959Z-001", "corrected_FA_image")

In [151]:
images_list = [nib.load(path).get_fdata() for i, path in enumerate(dataset["Path"])]
images = np.array(images_list, dtype='float32')
labels = dataset["Group"].to_numpy()

In [152]:
X_train, x_test, Y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=0)
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.21, random_state=1)

In [153]:
def make_model(shape=(110, 110, 110)):
  model = Sequential([
      
      Conv2D(8, (3,3), padding='same', input_shape=shape),
      BatchNormalization(),
      Activation('relu'),

      MaxPool2D((2,2), strides=2),
      #Dropout(0.4),

      Conv2D(16, (3,3), padding='same'),
      BatchNormalization(),
      Activation('relu'),

      MaxPool2D((2,2), strides=2),
      #Dropout(0.4),

      Conv2D(32, (3,3), padding='same'),
      BatchNormalization(),
      Activation('relu'),

      MaxPool2D((2,2), strides=2),
      #Dropout(0.4),

      Flatten(),

      Dense(10, activation='relu'),
      Dropout(0.2),
      Dense(1, activation='sigmoid')
     
  ])
  return model

model = make_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 110, 110, 8)       7928      
                                                                 
 batch_normalization_6 (Batc  (None, 110, 110, 8)      32        
 hNormalization)                                                 
                                                                 
 activation_6 (Activation)   (None, 110, 110, 8)       0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 55, 55, 8)        0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 55, 55, 16)        1168      
                                                                 
 batch_normalization_7 (Batc  (None, 55, 55, 16)      

In [154]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [155]:
reduce_on_plateau = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0)

epochs= 200
batch_size = 16 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    steps_per_epoch = len(x_train)/batch_size,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    validation_steps=len(x_val)/batch_size,
                    callbacks=[reduce_on_plateau])     

Epoch 1/200
4/4 [==============================] - 5s 584ms/step - loss: 0.7278 - accuracy: 0.5441 - val_loss: 0.6929 - val_accuracy: 0.6842 - lr: 0.0010
Epoch 2/200
4/4 [==============================] - 2s 488ms/step - loss: 0.6930 - accuracy: 0.4265 - val_loss: 0.6925 - val_accuracy: 0.6316 - lr: 0.0010
Epoch 3/200
4/4 [==============================] - 2s 490ms/step - loss: 0.6935 - accuracy: 0.4265 - val_loss: 0.6925 - val_accuracy: 0.6316 - lr: 0.0010
Epoch 4/200
4/4 [==============================] - 2s 459ms/step - loss: 0.6934 - accuracy: 0.4265 - val_loss: 0.6928 - val_accuracy: 0.6316 - lr: 0.0010
Epoch 5/200
4/4 [==============================] - 2s 476ms/step - loss: 0.6933 - accuracy: 0.4265 - val_loss: 0.6930 - val_accuracy: 0.6316 - lr: 0.0010
Epoch 6/200
4/4 [==============================] - 2s 439ms/step - loss: 0.6932 - accuracy: 0.4265 - val_loss: 0.6932 - val_accuracy: 0.3684 - lr: 0.0010
Epoch 7/200
4/4 [==============================] - 2s 450ms/step - loss: 0.6

KeyboardInterrupt: 